In [1]:
# Dependencies
import pandas as pd

# Save path to data set in a variable
file = "Resources/purchase_data.csv"

# Use Pandas to read data
purchase_data = pd.read_csv(file)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
#Total Number of Players
players = purchase_data["SN"].unique()
players_count = len(players)

pCount_table = pd.DataFrame({"Total Players": [players_count]})
pCount_table

,Total Players
0,576


In [3]:
### Purchasing Analysis (Total)
#Number of Unique Items
items_count = len(purchase_data["Item ID"].unique())

# Average Purchase Price
avg_price = purchase_data["Price"].mean()

# Total Number of Purchases
num_purchases = len(purchase_data["Price"])

# Total Revenue
total_revenue = purchase_data["Price"].sum()

purchasing_analysis = pd.DataFrame({"Number of Unique Items" : [items_count],
                                    "Average Price" : [avg_price],
                                    "Number of Purchases" : [num_purchases],
                                    "Total Revenue" : [total_revenue]})
purchasing_analysis = purchasing_analysis.round(2)

purchasing_analysis["Average Price"] = purchasing_analysis["Average Price"].map("${0:,.2f}".format)
purchasing_analysis["Total Revenue"] = purchasing_analysis["Total Revenue"].map("${0:,.2f}".format)

purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [4]:
### Gender Demographics
# Percentage and Count of Male Players
# Percentage and Count of Female Players
# Percentage and Count of Other / Non-Disclosed

gender_info = purchase_data.loc[:, ("SN", "Gender")]

gender_info.drop_duplicates(subset = ["SN"], keep = "last", inplace = True)

gender_counts = gender_info.groupby("Gender").size().reset_index(name='Total Count')
gender_percent = (gender_counts["Total Count"]/players_count)*100
gender_counts["Percentage of Players"] = gender_percent.map("{0:,.2f}%".format)

gender_counts

,Gender,Total Count,Percentage of Players
0,Female,81,14.06%
1,Male,484,84.03%
2,Other / Non-Disclosed,11,1.91%


In [5]:
### Purchasing Analysis (Gender)
#Purchase Count
purchases_by_gender = purchase_data.groupby(["Gender"]).size().reset_index(name="Purchase Count")

In [6]:
gender_purchases_df = pd.DataFrame(purchases_by_gender)

In [7]:
#Average Purchase Price
avg_purchases_by_gender = purchase_data.groupby(["Gender"]).mean()
avg_purchases_by_gender = avg_purchases_by_gender.rename(
    columns={"Price" : "Average Purchase Price"})

In [8]:
#Total Purchase Value
tot_purchases_by_gender = purchase_data.groupby(["Gender"]).sum()
tot_purchases_by_gender = tot_purchases_by_gender.rename(
    columns={"Price" : "Total Purchase Value"})

In [9]:
first_merge = pd.merge(tot_purchases_by_gender, avg_purchases_by_gender, on = "Gender", how="outer")
columns = ["Gender","Average Purchase Price", "Total Purchase Value"]
first_merge = first_merge[["Total Purchase Value", "Average Purchase Price"]]

In [10]:
second_merge = pd.merge(gender_purchases_df, first_merge, on="Gender", how="inner")

In [11]:
#Average Purchase Total per Person by Gender
avg_per_person = (second_merge[("Total Purchase Value")])/(gender_counts[("Total Count")])

In [12]:
second_merge["Avg Total Purchase per Person"] = avg_per_person.map("${0:,.2f}".format)
second_merge["Total Purchase Value"] = second_merge["Total Purchase Value"].map("${0:,.2f}".format)
second_merge["Average Purchase Price"] = second_merge["Average Purchase Price"].map("${0:,.2f}".format)

second_merge

,Gender,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
0,Female,113,$361.94,$3.20,$4.47
1,Male,652,"$1,967.64",$3.02,$4.07
2,Other / Non-Disclosed,15,$50.19,$3.35,$4.56
